##Setup

In [1]:
#connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U transformers
#!pip install vllm==0.5.3.post1
!pip install -U torch==2.5.1
!pip install --upgrade torchvision

!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 100.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [3]:
!pip install tqdm
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.5 MB/s eta 0:00:00


In [4]:

#get data file
!wget -q -O data.csv "https://docs.google.com/uc?export=download&id=13NuKWse5ZLbE7-3WtmdzYSeOOaJ3MydG"


In [5]:
import pandas as pd

data = pd.read_csv("data.csv")
#print(data.head())
print(data.columns)

Index(['id_1', 'id_2', 'text_1', 'text_2', '1_better', 'prompt'], dtype='object')


In [32]:
df = data.sample(n=100, random_state=42)
print(df.columns)

Index(['id_1', 'id_2', 'text_1', 'text_2', '1_better', 'prompt'], dtype='object')


In [33]:
prompt = '''
You are an essay grader comparing essays using the following rubric.
A high-scoring essay has native-like facility in the use of language with syntactic variety, appropriate word choice and phrases; well-controlled text organization; precise use of grammar and conventions; rare language inaccuracies that do not impede communication.
A low scoring essay has a limited range of familiar words or phrases loosely strung together; frequent errors in grammar (including syntax) and usage. Communication impeded in most cases by language inaccuracies.

Here is Essay A:
'''

In [34]:
rubric_overall= '''A high-scoring essay has native-like facility in the use of language with syntactic variety, appropriate word choice and phrases; well-controlled text organization; precise use of grammar and conventions; rare language inaccuracies that do not impede communication.
A low scoring essay has a limited range of familiar words or phrases loosely strung together; frequent errors in grammar (including syntax) and usage. Communication impeded in most cases by language inaccuracies.'''

rubric_cohesion = '''A high-scoring essay has text organization consistently well-controlled using a variety of effective linguistic features such as reference and transitional words and phrases to connect ideas across sentences and paragraphs; appropriate overlap of ideas.
A low scoring essay has no clear control of organization; cohesive devices not present or unsuccessfully used; presentation of ideas unclear'''

rubric_syntax = '''A high-scoring essay has flexible and effective use of a full range of syntactic structures including simple, compound, and complex sentences; there may be rare minor and negligible errors in sentence formation.
A low-scoring essay has pervasive and basic errors in sentence structure and word order that cause confusion; basic sentence errors are common.'''


rubric_vocab = '''A high-scoring essay has a wide range of vocabulary flexibly and effectively used to convey precise meanings; skillful use of topic-related terms and less common words; rare negligible inaccuracies in word use.
A low scoring essay has limited vocabulary often inappropriately used; limited control of word choice and word forms; little attempt to use topic-related terms.'''


rubric_phrase = '''A high scoring essay has flexible and effective use of a variety of phrases, such as idioms, collocations, and lexical bundles, to convey precise and subtle meanings; rare minor inaccuracies that are negligible.
A low scoring essay hsa memorized chunks of language, or simple phrasal patterns, predominate; many repetitions and misuses of phrases. '''


rubric_grammar = '''A high scoring essay has command of grammar and usage with few or no errors. A low scoring essay has errors in grammar and usage throughout.'''


rubric_conv = '''A high scoring essay has consistent use of appropriate conventions to convey meaning; spelling, capitalization, and punctuation errors nonexistent or negligible. A low scoring essay has minimal use of conventions; spelling, capitalization, and punctuation errors throughout.'''


##LLM grading

In [35]:
import torch

In [36]:
prompted_essays = []
for i, row in enumerate(df.iterrows()):
  row = row[1]
  prompted_essays.append(prompt + row['text_1'] + "\n and here is Essay B: \n" + row['text_2']+ "\n")

In [37]:
prompted_essays[0]

'\nYou are an essay grader comparing essays using the following rubric.\nA high-scoring essay has native-like facility in the use of language with syntactic variety, appropriate word choice and phrases; well-controlled text organization; precise use of grammar and conventions; rare language inaccuracies that do not impede communication.\nA low scoring essay has a limited range of familiar words or phrases loosely strung together; frequent errors in grammar (including syntax) and usage. Communication impeded in most cases by language inaccuracies.\n\nHere is Essay A:\nSuccesses comes in so many diffrent whays,but the real questchon is how can having a good attitude lead you to being more successesful in life. I think that starts with your attitude ,attitude plays a big part in your life, for example if your about to take a really big test in math class and you tell yourself befor the teast that your going to fail and your just having a negtive attitude about it, your more likely to fail

In [46]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import csv
from tqdm import tqdm

# Load model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
model = GPT2LMHeadModel.from_pretrained('gpt2-large')
model = model.to('cpu')  # Force CPU usage

def grade(prompt):
    # Restructure the prompt to encourage a clear A/B choice
    prompt_with_question = prompt + "\nAfter comparing both essays based on the rubric, the better essay is essay "

    # Tokenize
    inputs = tokenizer(prompt_with_question, return_tensors='pt', truncation=True, max_length=900)
    inputs = {k: v.to('cpu') for k, v in inputs.items()}

    # Generate just a few tokens to see if it's A or B
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=1,  # Just need one token to say A or B
            num_return_sequences=1,
            temperature=0.7,  # Add some randomness to avoid repetition
            return_dict_in_generate=True,
            output_scores=True  # We'll look at token probabilities
        )

    # Look at the probabilities of "A" vs "B" in the output
    token_id_A = tokenizer.encode(" A")[0]  # Space matters in tokenization
    token_id_B = tokenizer.encode(" B")[0]

    logits = output.scores[0][0]  # Get logits for first (only) generated token
    prob_A = torch.softmax(logits, dim=0)[token_id_A].item()
    prob_B = torch.softmax(logits, dim=0)[token_id_B].item()

    # Determine answer based on higher probability
    answer = "A" if prob_A > prob_B else "B"
    confidence = max(prob_A, prob_B)

    return {
        'prediction': answer,
        'confidence': confidence,
        'A_probability': prob_A,
        'B_probability': prob_B
    }

# Process all essays and write to CSV
output_path = '/content/drive/MyDrive/senior_thesis/part_2/outputs/gpt2-large.csv'

with open(output_path, 'w', newline='') as csvfile:
    fieldnames = ['id_1', 'id_2', 'prompt', 'prediction']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for i, essay_prompt in enumerate(tqdm(prompted_essays, desc="Grading essays")):
        try:
            result = grade(essay_prompt)

            # Write to CSV
            writer.writerow({
                'id_1': df.iloc[i]['id_1'],
                'id_2': df.iloc[i]['id_2'],
                'prompt': df.iloc[i]['prompt'],
                'prediction': result['prediction']
            })

            # Print progress
            if i % 10 == 0:
                print(f"Essay {i}: Predicted {result['prediction']} (Confidence: {result['confidence']:.4f})")

        except Exception as e:
            print(f"Error processing essay {i}: {e}")
            writer.writerow({
                'id_1': df.iloc[i]['id_1'],
                'id_2': df.iloc[i]['id_2'],
                'prompt': df.iloc[i]['prompt'],
                'prediction': 'Error'
            })

print(f"Results written to {output_path}")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

...

Essay 80: Predicted A (Confidence: 0.0000)


Grading essays:  91%|█████████ | 91/100 [04:10<00:24,  2.73s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Essay 90: Predicted A (Confidence: 0.1933)


Grading essays: 100%|██████████| 100/100 [04:36<00:00,  2.76s/it]

Results written to /content/drive/MyDrive/senior_thesis/part_2/outputs/gpt2-large.csv


In [44]:
results

[{'prediction': 'A',
  'confidence': 0.0001376653672195971,
  'A_probability': 0.0001376653672195971,
  'B_probability': 3.8355665310518816e-05,
  'essay_index': 0},
 {'prediction': 'A',
  'confidence': 4.9701338866725564e-05,
  'A_probability': 4.9701338866725564e-05,
  'B_probability': 1.3142042007530108e-05,
  'essay_index': 1},
 {'prediction': 'B',
  'confidence': 1.0639269930834416e-05,
  'A_probability': 9.147513992502354e-06,
  'B_probability': 1.0639269930834416e-05,
  'essay_index': 2},
 {'prediction': 'A',
  'confidence': 2.664785938577552e-07,
  'A_probability': 2.664785938577552e-07,
  'B_probability': 1.7246737371579002e-08,
  'essay_index': 3},
 {'prediction': 'A',
  'confidence': 5.417131251306273e-05,
  'A_probability': 5.417131251306273e-05,
  'B_probability': 2.0892712200293317e-05,
  'essay_index': 4},
 {'prediction': 'A',
  'confidence': 7.2482062023482285e-06,
  'A_probability': 7.2482062023482285e-06,
  'B_probability': 4.611706572177354e-06,
  'essay_index': 5},


In [40]:
import numpy as np
import csv
gpt = np.array(gpt_output)

if gpt.ndim == 1:
    gpt = gpt.reshape(-1, 1)

with open('/content/drive/MyDrive/senior_thesis/part_2/outputs/gpt2.csv', 'w', newline='') as out_file:
    out_file.write('id_1,id_2,prompt,pred\n')

    # writer = csv.writer(f)
    for i, item in enumerate(gpt):
        id_1 = df['id_1'].to_list()
        id_2 = df['id_2'].to_list()
        prompt = df['prompt'].to_list()
        print(item[0])
        out_file.write(f'{id_1[i]},{id_2[i]},{prompt[i]},{item[0]}\n')


{'generated_text': '\nYou are an essay grader comparing essays using the following rubric.\nA high-scoring essay has native-like facility in the use of language with syntactic variety, appropriate word choice and phrases; well-controlled text organization; precise use of grammar and conventions; rare language inaccuracies that do not impede communication.\nA low scoring essay has a limited range of familiar words or phrases loosely strung together; frequent errors in grammar (including syntax) and usage. Communication impeded in most cases by language inaccuracies.\n\nHere is Essay A:\nSuccesses comes in so many diffrent whays,but the real questchon is how can having a good attitude lead you to being more successesful in life. I think that starts with your attitude ,attitude plays a big part in your life, for example if your about to take a really big test in math class and you tell yourself befor the teast that your going to fail and your just having a negtive attitude about it, your 